In [1]:
from typing import Tuple, Dict, List
import json
from itertools import chain
import pandas as pd
from sklearn.linear_model import LogisticRegression


def get_model_params(df: pd.DataFrame, ordering: List[List[str]], solver='liblinear', penalty='l1', C=0.2) -> pd.DataFrame:
    """
    Gets LASSO regression parameters for each variable.

    :param df: Data.
    :param solver: Solver (liblinear or saga). Default: `liblinear`.
    :param penalty: Penalty. Default: `l1`.
    :param C: Regularlization. Default: `0.2`.
    :return: LASSO regression parameters for each variable.
    """

    def get_model(df, X_cols, y_col):
        X = df[X_cols]
        y = df[y_col]

        model = LogisticRegression(penalty=penalty, solver=solver, C=C)
        model.fit(X, y)

        return model

    def extract_model_params(y, fields, model):
        child = {'child': y}
        intercepts = {'intercept': model.intercept_[0]}
        coefs = {field: coef for field, coef in zip(fields, model.coef_[0])}
        others = {field: 0.0 for field in fields[len(coefs):]}

        p = {**child, **intercepts}
        p = {**p, **coefs}
        p = {**p, **others}

        return p
    
    def get_models(i, y_cols):
        X_cols = list(chain(*ordering[:i]))
        return [(y_col, get_model(df, X_cols, y_col)) for y_col in y_cols]

    models = chain(*[get_models(i, y_cols) for i, y_cols in enumerate(ordering) if i > 0])
    param_df = pd.DataFrame([extract_model_params(y, df.columns, model) for y, model in models])
    return param_df

In [2]:
df = pd.read_csv('../data/data-binary.csv')

with open('../data/data-binary-complete.json', 'r') as f:
    meta = json.load(f)
ordering = meta['ordering']

get_model_params(df, ordering)

b = ['a']
c = ['a', 'b']
d = ['a', 'b', 'c']
e = ['a', 'b', 'c', 'd']


,child,intercept,a,b,c,d,e
0,b,-1.401360,0.000000,0.000000,0.0,0.000000,0.0
1,c,-1.402283,0.010293,2.667348,0.0,0.000000,0.0
2,d,-1.256557,-0.099824,-0.085225,0.0,0.000000,0.0
3,e,-2.245915,0.000000,0.028802,0.0,4.337575,0.0


In [3]:
df = pd.read_csv('../data/data-binary.csv')

with open('../data/data-binary-partial.json', 'r') as f:
    meta = json.load(f)
ordering = meta['ordering']

get_model_params(df, ordering)

c = ['a', 'b']
d = ['a', 'b']
e = ['a', 'b', 'c', 'd']


,child,intercept,a,b,c,d,e
0,c,-1.402075,0.010107,2.667314,0.0,0.000000,0.0
1,d,-1.257045,-0.099291,-0.085160,0.0,0.000000,0.0
2,e,-2.245821,0.000000,0.028780,0.0,4.337819,0.0
